To run this notebook SentenceTransformer needed to be installed

In [1]:
#!pip install SentenceTransformer

## Import Libraries

In [2]:
'''basics'''
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)
import vectorize_embed as em
import pandas as pd
#import pickle5 as pickle
import pickle
import numpy as np


'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

'''features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB


'''Metrics/Evaluation'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, multi_labeled
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings('ignore')

import joblib

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


## Import data

In [3]:
df = pd.read_csv('../../data/processed/encoded_labels/technical_area_1.csv')
df.columns

Index(['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'technical_area_1',
       '_access_and_benefit_sharing', '_agrobiodiversity',
       '_biodiversity_financing', '_ecosystem_based_adaptation',
       '_ecosystem_based_mitigation', '_small_island_dev_states_program_sids',
       '_wildlife_conservation', 'climate_change_adaptation_programme',
       'climate_information_and_early_warning_systems_ci_ews',
       'climate_resilient_access_to_water_supply_sanitation',
       'climate_resilient_integrated_water_resource_coastal_area_management',
       'cross_cutting_governance_and_learning',
       'cross_sectoral_climate_resilient_livelihoods',
       'ecosystem_based_adaptation', 'ecosystem_management_and_restoration',
       'ecosystems_and_biodiversity_programme', 'energy_access',
       'energy_efficiency', 'environmental_governance_programme',
       'fostering_food_security_and_resilient_agricultural_systems',
       'heavy_metals', 'mainstreaming_biodiversity',
       'm

# Compare different embeddings performances

In [4]:
categories = ['_access_and_benefit_sharing', '_agrobiodiversity',
       '_biodiversity_financing', '_ecosystem_based_adaptation',
       '_ecosystem_based_mitigation', '_small_island_dev_states_program_sids',
       '_wildlife_conservation', 'climate_change_adaptation_programme',
       'climate_information_and_early_warning_systems_ci_ews',
       'climate_resilient_access_to_water_supply_sanitation',
       'climate_resilient_integrated_water_resource_coastal_area_management',
       'cross_cutting_governance_and_learning',
       'cross_sectoral_climate_resilient_livelihoods',
       'ecosystem_based_adaptation', 'ecosystem_management_and_restoration',
       'ecosystems_and_biodiversity_programme', 'energy_access',
       'energy_efficiency', 'environmental_governance_programme',
       'fostering_food_security_and_resilient_agricultural_systems',
       'heavy_metals', 'mainstreaming_biodiversity',
       'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
       'miscellaneous_energy_program', 'no tag', 'ozone_depleting_substances',
       'persistent_organic_pollutants',
       'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
       'renewable_energy', 'strengthening_conservation_areas',
       'sustainable_management_of_oceans_in_a_changing_climate',
       'waste_management']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()


## Train multiple Embeddings with SGD / OneVsRest multi-label strategy

In [5]:
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    



In [6]:
from tabulate import tabulate  
header = lis[0].keys()
rows =  [x.values() for x in lis]

<table>
<thead>
<tr><th>Category                                                                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>_access_and_benefit_sharing                                                               </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>_agrobiodiversity                                                                         </td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.994475138121547    
 0.988950276243094    
 0.9834254143646409    
 0.9723756906077348                 </td><td>0.5   
 0.5   
 0.5   
 0.5                  </td><td>0.497237569

### copy-past the output of the last cell here


In [9]:
print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category                                                                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>_access_and_benefit_sharing                                                               </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>_agrobiodiversity                                                                         </td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.994475138121547    
 0.988950276243094    
 0.9834254143646409    
 0.9723756906077348                 </td><td>0.5   
 0.5   
 0.5   
 0.5                  </td><td>0.497237569

In [ ]:
<table>
<thead>
<tr><th>Category                                                                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>_access_and_benefit_sharing                                                               </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>_agrobiodiversity                                                                         </td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.994475138121547    
 0.988950276243094    
 0.9834254143646409    
 0.9723756906077348                 </td><td>0.5   
 0.5   
 0.5   
 0.5                  </td><td>0.4972375690607735    
 0.494475138121547    
 0.49171270718232046    
 0.4861878453038674               </td><td>0.4986149584487534    
 0.49722222222222223    
 0.49582172701949867    
 0.4929971988795518           </td></tr>
<tr><td>_biodiversity_financing                                                                   </td><td>Distilbert    
 Roberta    
 Bert    
 Glove              </td><td>1.0    
 1.0    
 1.0    
 0.994475138121547                 </td><td>1.0   
 1.0   
 1.0   
 0.5                  </td><td>1.0    
 1.0    
 1.0    
 0.4972375690607735               </td><td>1.0    
 1.0    
 1.0    
 0.4986149584487534           </td></tr>
<tr><td>_ecosystem_based_adaptation                                                               </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.988950276243094    
 0.988950276243094    
 0.988950276243094    
 0.988950276243094                 </td><td>0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223                  </td><td>0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223               </td><td>0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223           </td></tr>
<tr><td>_ecosystem_based_mitigation                                                               </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>_small_island_dev_states_program_sids                                                     </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>_wildlife_conservation                                                                    </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>climate_change_adaptation_programme                                                       </td><td>Bert    
 Roberta    
 Distilbert    
 Glove              </td><td>0.9779005524861878    
 0.9723756906077348    
 0.9723756906077348    
 0.9226519337016574                 </td><td>0.9166666666666667   
 0.9   
 0.9223214285714285   
 0.7935192953020134                  </td><td>0.9875776397515528    
 0.984472049689441    
 0.9406832298136646    
 0.9346273291925465               </td><td>0.948256146369354    
 0.9365580091132142    
 0.931236228250133    
 0.8428039702233252           </td></tr>
<tr><td>climate_information_and_early_warning_systems_ci_ews                                      </td><td>Distilbert    
 Bert    
 Glove    
 Roberta              </td><td>0.9779005524861878    
 0.9613259668508287    
 0.9779005524861878    
 0.9613259668508287                 </td><td>0.7416201117318435   
 0.5914772727272727   
 0.4889502762430939   
 0.4887640449438202                  </td><td>0.6221751412429379    
 0.6137005649717514    
 0.5    
 0.4915254237288136               </td><td>0.6610486891385767    
 0.6011960969468052    
 0.494413407821229    
 0.49014084507042255           </td></tr>
<tr><td>climate_resilient_access_to_water_supply_sanitation                                       </td><td>Roberta    
 Distilbert    
 Bert    
 Glove              </td><td>0.9834254143646409    
 0.9834254143646409    
 0.9834254143646409    
 0.9779005524861878                 </td><td>0.744413407821229   
 0.49171270718232046   
 0.49171270718232046   
 0.49166666666666664                  </td><td>0.6638576779026217    
 0.5    
 0.5    
 0.49719101123595505               </td><td>0.6957983193277311    
 0.49582172701949867    
 0.49582172701949867    
 0.49441340782122906           </td></tr>
<tr><td>climate_resilient_integrated_water_resource_coastal_area_management                       </td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.9392265193370166    
 0.9337016574585635    
 0.9392265193370166    
 0.9337016574585635                 </td><td>0.599483204134367   
 0.5480491329479769   
 0.48295454545454547   
 0.4828571428571429                  </td><td>0.6466666666666666    
 0.5633333333333334    
 0.4857142857142857    
 0.4828571428571429               </td><td>0.6174831892411142    
 0.5541871921182266    
 0.4843304843304843    
 0.4828571428571428           </td></tr>
<tr><td>cross_cutting_governance_and_learning                                                     </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>cross_sectoral_climate_resilient_livelihoods                                              </td><td>Glove    
 Roberta    
 Distilbert    
 Bert              </td><td>0.9337016574585635    
 0.9337016574585635    
 0.9447513812154696    
 0.9281767955801105                 </td><td>0.6909638554216868   
 0.6492248062015504   
 0.6441947565543071   
 0.4745762711864407                  </td><td>0.8071705426356589    
 0.6492248062015504    
 0.5497416020671835    
 0.4883720930232558               </td><td>0.7322485207100591    
 0.6492248062015504    
 0.569047619047619    
 0.48137535816618904           </td></tr>
<tr><td>ecosystem_based_adaptation                                                                </td><td>Glove    
 Distilbert    
 Bert    
 Roberta              </td><td>1.0    
 1.0    
 1.0    
 0.994475138121547                 </td><td>1.0   
 1.0   
 1.0   
 0.5                  </td><td>1.0    
 1.0    
 1.0    
 0.4972375690607735               </td><td>1.0    
 1.0    
 1.0    
 0.4986149584487534           </td></tr>
<tr><td>ecosystem_management_and_restoration                                                      </td><td>Bert    
 Roberta    
 Distilbert    
 Glove              </td><td>0.8066298342541437    
 0.8232044198895028    
 0.6464088397790055    
 0.8674033149171271                 </td><td>0.6507839721254356   
 0.6472315436241611   
 0.6284577723378213   
 0.43370165745856354                  </td><td>0.7296974522292994    
 0.6863057324840764    
 0.7785297239915074    
 0.5               </td><td>0.6718466718466719    
 0.6619981325863679    
 0.5821067821067822    
 0.4644970414201184           </td></tr>
<tr><td>ecosystems_and_biodiversity_programme                                                     </td><td>Bert    
 Roberta    
 Glove    
 Distilbert              </td><td>0.988950276243094    
 0.9834254143646409    
 0.9779005524861878    
 0.9779005524861878                 </td><td>0.9943820224719101   
 0.9916201117318435   
 0.9888888888888889   
 0.9888888888888889                  </td><td>0.8    
 0.7    
 0.6    
 0.6               </td><td>0.8721751412429377    
 0.7814889336016098    
 0.6610486891385767    
 0.6610486891385767           </td></tr>
<tr><td>energy_access                                                                             </td><td>Bert    
 Glove    
 Distilbert    
 Roberta              </td><td>0.9613259668508287    
 0.9613259668508287    
 0.9668508287292817    
 0.9558011049723757                 </td><td>0.6371100164203612   
 0.5914772727272727   
 0.4888268156424581   
 0.4887005649717514                  </td><td>0.7358757062146892    
 0.6137005649717514    
 0.4943502824858757    
 0.4887005649717514               </td><td>0.6718466718466718    
 0.6011960969468052    
 0.49157303370786515    
 0.4887005649717514           </td></tr>
<tr><td>energy_efficiency                                                                         </td><td>Distilbert    
 Glove    
 Bert    
 Roberta              </td><td>0.9668508287292817    
 0.9613259668508287    
 0.9613259668508287    
 0.9558011049723757                 </td><td>0.8772609819121446   
 0.8244575936883629   
 0.8383040935672514   
 0.8064171122994652                  </td><td>0.8122994652406417    
 0.8518716577540106    
 0.8093582887700534    
 0.8064171122994652               </td><td>0.8412280701754387    
 0.8375016031807105    
 0.8230694037145649    
 0.8064171122994652           </td></tr>
<tr><td>environmental_governance_programme                                                        </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.9723756906077348    
 0.9558011049723757    
 0.9558011049723757                 </td><td>0.9615384615384616   
 0.985632183908046   
 0.8024880708929789   
 0.8353801169590642                  </td><td>0.9970414201183432    
 0.7916666666666667    
 0.9376232741617356    
 0.7827909270216963               </td><td>0.9785163204747775    
 0.8611324228939696    
 0.8546184738955824    
 0.8064171122994652           </td></tr>
<tr><td>fostering_food_security_and_resilient_agricultural_systems                                </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>heavy_metals                                                                              </td><td>Roberta    
 Bert    
 Distilbert    
 Glove              </td><td>0.994475138121547    
 0.9723756906077348    
 0.988950276243094    
 0.9779005524861878                 </td><td>0.9972067039106145   
 0.6875   
 0.9944444444444445   
 0.6610486891385767                  </td><td>0.8333333333333333    
 0.9859550561797753    
 0.6666666666666666    
 0.6610486891385767               </td><td>0.8985994397759104    
 0.7656047656047655    
 0.7472067039106145    
 0.6610486891385767           </td></tr>
<tr><td>mainstreaming_biodiversity                                                                </td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.9281767955801105    
 0.8784530386740331    
 0.8342541436464088    
 0.9558011049723757                 </td><td>0.6131656804733727   
 0.5935897435897436   
 0.569000819000819   
 0.48055555555555557                  </td><td>0.6884236453201971    
 0.7996715927750411    
 0.7766830870279147    
 0.49712643678160917               </td><td>0.638944299524321    
 0.6229166666666667    
 0.5784161490683231    
 0.48870056497175146           </td></tr>
<tr><td>mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development</td><td>Glove    
 Bert    
 Distilbert    
 Roberta              </td><td>0.9226519337016574    
 0.9281767955801105    
 0.9171270718232044    
 0.9226519337016574                 </td><td>0.7709551656920077   
 0.8432080924855492   
 0.7678362573099415   
 0.8630681818181818                  </td><td>0.7991391678622668    
 0.6703730272596844    
 0.6642754662840746    
 0.6145982783357246               </td><td>0.7840831629175189    
 0.7207121661721068    
 0.6998341625207297    
 0.6612299465240642           </td></tr>
<tr><td>miscellaneous_energy_program                                                              </td><td>Glove    
 Bert    
 Distilbert    
 Roberta              </td><td>0.9834254143646409    
 0.9834254143646409    
 0.9834254143646409    
 0.9779005524861878                 </td><td>0.744413407821229   
 0.744413407821229   
 0.49171270718232046   
 0.49166666666666664                  </td><td>0.6638576779026217    
 0.6638576779026217    
 0.5    
 0.49719101123595505               </td><td>0.6957983193277311    
 0.6957983193277311    
 0.49582172701949867    
 0.49441340782122906           </td></tr>
<tr><td>no tag                                                                                    </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.988950276243094    
 0.988950276243094    
 0.988950276243094    
 0.988950276243094                 </td><td>0.7472067039106145   
 0.494475138121547   
 0.494475138121547   
 0.494475138121547                  </td><td>0.7472067039106145    
 0.5    
 0.5    
 0.5               </td><td>0.7472067039106145    
 0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223           </td></tr>
<tr><td>ozone_depleting_substances                                                                </td><td>Distilbert    
 Glove    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.988950276243094    
 0.988950276243094    
 0.988950276243094                 </td><td>0.9972222222222222   
 0.494475138121547   
 0.494475138121547   
 0.494475138121547                  </td><td>0.75    
 0.5    
 0.5    
 0.5               </td><td>0.8319405756731661    
 0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223           </td></tr>
<tr><td>persistent_organic_pollutants                                                             </td><td>Distilbert    
 Roberta    
 Bert    
 Glove              </td><td>0.9723756906077348    
 0.9668508287292817    
 0.9668508287292817    
 0.9613259668508287                 </td><td>0.8886363636363637   
 0.8219047619047619   
 0.8608757062146892   
 0.9805555555555556                  </td><td>0.7471098265895953    
 0.7442196531791907    
 0.6846098265895953    
 0.5625               </td><td>0.8005289839100727    
 0.7770935960591133    
 0.7414285714285713    
 0.6011960969468051           </td></tr>
<tr><td>protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate           </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.9834254143646409    
 0.9723756906077348    
 0.9723756906077348    
 0.9613259668508287                 </td><td>0.9916201117318435   
 0.4861878453038674   
 0.4861878453038674   
 0.4860335195530726                  </td><td>0.7    
 0.5    
 0.5    
 0.4943181818181818               </td><td>0.7814889336016098    
 0.4929971988795518    
 0.4929971988795518    
 0.49014084507042255           </td></tr>
<tr><td>renewable_energy                                                                          </td><td>Glove    
 Bert    
 Distilbert    
 Roberta              </td><td>0.9668508287292817    
 0.9613259668508287    
 0.9502762430939227    
 0.9226519337016574                 </td><td>0.9828571428571429   
 0.8743540051679586   
 0.8158914728682171   
 0.6614884393063584                  </td><td>0.75    
 0.7857495069033531    
 0.7411242603550297    
 0.610207100591716               </td><td>0.8246124031007751    
 0.8230694037145649    
 0.7725178047758694    
 0.62953216374269           </td></tr>
<tr><td>strengthening_conservation_areas                                                          </td><td>Distilbert    
 Bert    
 Roberta    
 Glove              </td><td>0.9226519337016574    
 0.9005524861878453    
 0.8839779005524862    
 0.9116022099447514                 </td><td>0.7707602339181286   
 0.6843455945252352   
 0.6335341365461847   
 0.4558011049723757                  </td><td>0.6753787878787878    
 0.6632575757575758    
 0.6259469696969697    
 0.5               </td><td>0.7099358974358975    
 0.6728915662650603    
 0.6295682681999805    
 0.476878612716763           </td></tr>
<tr><td>sustainable_management_of_oceans_in_a_changing_climate                                    </td><td>Glove    
 Distilbert    
 Bert    
 Roberta              </td><td>0.9779005524861878    
 0.9779005524861878    
 0.9779005524861878    
 0.9723756906077348                 </td><td>0.7443502824858756   
 0.7416201117318435   
 0.7416201117318435   
 0.4888888888888889                  </td><td>0.7443502824858756    
 0.6221751412429379    
 0.6221751412429379    
 0.4971751412429379               </td><td>0.7443502824858756    
 0.6610486891385767    
 0.6610486891385767    
 0.4929971988795518           </td></tr>
<tr><td>waste_management                                                                          </td><td>Roberta    
 Bert    
 Glove    
 Distilbert              </td><td>1.0    
 1.0    
 0.994475138121547    
 0.994475138121547                 </td><td>1.0   
 1.0   
 0.4972375690607735   
 0.4972375690607735                  </td><td>1.0    
 1.0    
 0.5    
 0.5               </td><td>1.0    
 1.0    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
</tbody>
</table>
1
​
